# Objective
### Building a geospatial data analysis system that will:

- Load US map and Hurricane Florence data

- Process and analyze the storm path

- Visualize it meaningfully

- Produce exportable results (like annotated maps)



In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt

In [5]:
! cd /content/drive/MyDrive/githubrepos/geospatial-florence-project-root/geospatial-florence-project/data/ && ls

florence_2018_data.py  florence_2018.json


In [7]:
data_path = "/content/drive/MyDrive/githubrepos/geospatial-florence-project-root/geospatial-florence-project/data/florence_2018.json"

In [21]:
path = "/content/drive/MyDrive/githubrepos/geospatial-florence-project-root/geospatial-florence-project/data/"

In [25]:
import json
from pathlib import Path

def cyclone_json_to_geojson(
    input_path: str | Path,
    output_path: str | Path,
    lat_key: str = "Lat",
    lon_key: str = "Long"
) -> Path:
    """
    Convert a cyclone advisory JSON (list of dicts) to a
    GeoJSON FeatureCollection of Point geometries.

    Parameters
    ----------
    input_path : str | Path
        Path to the raw advisory JSON file you showed.
    output_path : str | Path, default "cyclone_data.geojson"
        Where to write the GeoJSON file.
    lat_key, lon_key : str, default "Lat", "Long"
        Keys in each advisory dict that hold latitude and longitude.

    Returns
    -------
    Path
        The path to the written GeoJSON file.
    """
    input_path = Path(input_path)
    output_path = Path(output_path) / "florence_2018.geojson" # Modified this line

    # ---------- 1. Load the raw advisory list ----------
    with input_path.open() as f:
        advisories = json.load(f)

    # ---------- 2. Wrap each advisory in a GeoJSON Feature ----------
    features = []
    for item in advisories:
        try:
            # Extract coords and convert to float
            lat = float(item[lat_key])
            lon = float(item[lon_key])

            # Build GeoJSON Feature
            feature = {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [lon, lat]   # GeoJSON = [x, y] = [lon, lat]
                },
                # Everything *except* Lat/Long becomes properties
                "properties": {k: v for k, v in item.items() if k not in {lat_key, lon_key}}
            }
            features.append(feature)
        except (KeyError, ValueError):
            # Skip rows with missing or malformed coordinates
            continue

    # ---------- 3. Assemble FeatureCollection ----------
    geojson = {
        "type": "FeatureCollection",
        "features": features
    }

    # ---------- 4. Write to file ----------
    with output_path.open("w") as f:
        json.dump(geojson, f)

    return output_path

In [28]:
# Convert the file you showed (e.g. "florence_raw.json")
geojson_file = cyclone_json_to_geojson(input_path=data_path, output_path=path)